### Web Search Agent 2.

## LangChain tool-based websearch agent with LLM Fallback.

This is a huge upgrade over the previous agent because:
- DuckduckGo based websearch is now wrapped as a `tool` in langchain.
- If the websearch proves to be insufficient or inaccurate, there is a direct fallback to an LLM call (gemini)
- Error handling is present, and extensibility to using other tools is easy

In [15]:
from langchain.tools import Tool
from langchain.agents import initialize_agent, AgentType
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from ddgs import DDGS
from dotenv import load_dotenv
import os

load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0.7
)

### Defining the DuckDuckGo Web Search Tool

This is wrapped as a LangChain Tool for agent orchestration.

In [16]:
def ddg_search_tool(query: str) -> str:
    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, region="wt-wt", safesearch="Moderate", max_results=5)
            lines = []
            for r in results:
                lines.append(f"{r['title']}\n{r['href']}\n{r.get('body','')}\n")
            if not lines:
                return "No results found on DuckDuckGo."
            return "\n".join(lines)
    except Exception as e:
        return f"DuckDuckGo search failed: {str(e)}"

web_search = Tool(
    name="DuckDuckGo Web Search",
    func=ddg_search_tool,
    description="Use this tool for web searches. Input should be a concise query."
)

### Defining the fallback tool to answer using LLM

If web search is unhelpful, fallback to answering using the LLM by directly prompting.

In [ ]:
def answer_with_llm(query: str) -> str:
    prompt = f"Answer the following question in a helpful manner:\n{query}"
    try:
        return llm(prompt)
    except Exception as e:
        return f"LLM fallback failed: {str(e)}"

llm_fallback = Tool(
    name="Groq LLM Fallback",
    func=answer_with_llm,
    description="If web search fails, answer with your own knowledge."
)

### Initializing the Agent

The agent will:
- Try web search first
- If results are insufficient or an error occurs, fallback to the LLM for an answer

We use LangChain's AgentType.ZERO_SHOT_REACT_DESCRIPTION for tool selection.

In [ ]:
tools = [web_search, llm_fallback]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True, 
    verbose=True
)

### Testing the Agent

first query:
- Standard Web search on latest RAG research
- One tool call should be enough
  
Second query:
- Difference between a unicorn and time travelling fridge in machine learning
- Forces an LLM call directly without calling the search API because it can see that these concepts are made up.


In [23]:
print("=== Test 1: Standard Web Search ===")
response1 = agent.run("Latest research on retrieval augmented generation in 2024")
print(response1)

print("\n=== Test 2: Obscure Query (forces LLM Fallback) ===")
response2 = agent.run("Explain the difference between a unicorn and a time-traveling refrigerator in machine learning.")
print(response2)


=== Test 1: Standard Web Search ===


> Entering new AgentExecutor chain...
To find the latest research on retrieval augmented generation in 2024, I should first attempt to search the web for relevant information.

Action: DuckDuckGo Web Search
Action Input: retrieval augmented generation 2024 research
Observation: Wikipedia Retrieval-augmented generation - Wikipedia
https://en.wikipedia.org/wiki/Retrieval-augmented_generation
3 days ago - Retrieval-augmented generation (RAG) is a technique that enables large language models (LLMs) to retrieve and incorporate new information . With RAG, LLMs do not respond to user queries until they refer to a specified set of documents. These documents supplement information from the LLM's ...

[2312.10997] Retrieval - Augmented Generation for Large Language...
https://arxiv.org/abs/2312.10997
Retrieval - Augmented Generation (RAG) has emerged as a promising solution by incorporating knowledge from external databases.View a PDF of the paper titled Ret